In [22]:
import torch
from dataloader import *
from models.vgg import VGG11
from utils import *
import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model('./checkpoints/model.th', device)

In [3]:
model

VGG11(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

In [4]:
normal, targets = load_samples('./data/normal', 'fgsm')
adv, targets_ = load_samples('./data/adversarial', 'fgsm')

0  examples are loaded
50  examples are loaded
0  examples are loaded
50  examples are loaded


In [5]:
assert torch.eq(targets, targets_).sum() == 0 # initial prediction must be different from final prediction

In [16]:
normal_out = batch_model_execute(model, layer_ct=9, data=normal, device=device)

In [17]:
normal_out.shape

torch.Size([100, 512, 1, 1])

In [21]:
for nr in range(normal_out.shape[1]):
    activations = normal_out[:, nr, ...].flatten()
    plt.hist(activations)
    break

NameError: name 'plt' is not defined